In [1]:
import os
import numpy as np 
import re
import io
from utilities.features_csv import to_csv
import pandas as pd
from IPython.display import display
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize

In [2]:
import re 
import csv 
docs = {}
labels = {}
#I only work if you run me in the same folder as the actual text files 

with open('training_labels.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            id = int(row[1])
            score= float(row[2])
            labels[id]=score
            line_count += 1
i = 0
for root, dirs, files in os.walk("corpus/"):
    for file in files:
        if file.endswith(".txt"):
            id = re.sub("[^0-9]", "", file)
            id = int(id) 
            path_file = os.path.join(root,file)
            curdir = path_file
            file = open(curdir, 'r', errors='ignore')
            i+=1
            text = file.read()
            docs[id] = text

        

In [653]:
# #auto features here 
# import os 
# import numpy as np 
# import os
# import re 
# import csv 
# import math 
# from sklearn import svm
# from sklearn.svm import SVC
# from sklearn.model_selection import GridSearchCV
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer
# from nltk import word_tokenize
# import pandas as pd 
# train_x,train_y = [],[]
# for k in labels:
#     try:
#         if docs[k] and labels[k]:
#             train_x.append(docs[k]) 
#             train_y.append(labels[k])
#     except KeyError:
#         pass
# from sklearn.feature_extraction.text import CountVectorizer

# def vectorize(txt):
#     count_vect = CountVectorizer(stop_words=stopwords.words('english'),max_features = 6,strip_accents ='ascii')
#     #print(count_vect.get_feature_names)
#     vects = count_vect.fit_transform(txt)
#     print(vects.shape) 
#     return count_vect

# from sklearn.preprocessing import normalize 
# def normalize_vect(v):
#     return normalize(v)
    
# v = vectorize(train_x)
# imp_words = v.get_feature_names()
# id_ct = {}
# for id in docs:
#     id_ct[id] = [0]*len(imp_words)
# for id in id_ct:
#     txt = docs[id] 
#     for x in range(len(imp_words)):
#         word = imp_words[x]
#         if word in txt:
#             id_ct[id][x] +=1
# dat = pd.DataFrame(data = id_ct)
# dat = dat.transpose()
# dat.columns = imp_words
# dat['iD'] = dat.index

In [654]:
# dat.head()

In [684]:
data = pd.read_csv("final.csv")
frames = [data,dat]
data =  data.merge(dat,how='left',on='iD')
display(data.head(n=10))

,iD,minor,geo-location,contact_email,vendors,sell_personal,share_personal,Cookies,fog_index,avg_sentence_length,flesch_reading_ease,dale_chall_readability_score,Score,data,information,may,personal,privacy,use
0,20481,1,0,0,1,0,0,0,75.245659,183.114146,-4.42,9.08,3.0,1,1,1,1,1,1
1,20484,0,0,1,0,0,1,1,58.618557,141.546392,37.79,7.02,3.0,1,1,1,1,1,1
2,854,0,0,0,0,0,0,0,55.264865,133.162162,46.30,6.60,2.0,1,1,1,1,1,1
3,20490,1,0,1,0,0,0,1,70.405195,171.012987,7.88,8.48,3.0,1,1,1,1,1,1
4,20493,0,0,0,0,0,0,0,81.172242,197.930605,-10.99,9.82,2.0,1,1,1,0,1,1
5,173,0,0,1,0,0,0,1,47.731507,114.328767,65.41,5.67,3.0,1,1,1,1,1,1
6,11096,0,0,1,0,0,0,0,49.727168,119.317919,60.35,5.92,2.0,1,1,1,1,1,1
7,20500,0,0,0,0,0,0,1,92.804233,227.010582,-48.97,11.26,2.0,1,1,1,1,1,1
8,516,0,1,0,0,0,0,1,66.037158,160.092896,18.96,7.94,2.0,1,1,1,1,1,1
9,20506,0,0,0,0,0,0,0,42.903529,102.258824,77.66,5.07,2.0,0,1,1,1,1,1


In [694]:
#spliting labels and features
scores = data['Score']
features_raw = data.drop(['Score', 'iD'] , axis = 1)
features_raw = features_raw.drop(["sell_personal",'geo-location'],axis=1)
features_raw = features_raw.drop(['data','information','use','privacy','personal'],axis=1)
features_raw.head()

,minor,contact_email,vendors,share_personal,Cookies,fog_index,avg_sentence_length,flesch_reading_ease,dale_chall_readability_score,may
0,1,0,1,0,0,75.245659,183.114146,-4.42,9.08,1
1,0,1,0,1,1,58.618557,141.546392,37.79,7.02,1
2,0,0,0,0,0,55.264865,133.162162,46.30,6.60,1
3,1,1,0,0,1,70.405195,171.012987,7.88,8.48,1
4,0,0,0,0,0,81.172242,197.930605,-10.99,9.82,1


In [695]:
from sklearn.preprocessing import MinMaxScaler
# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler()
numerical = ['fog_index', 'avg_sentence_length', 'flesch_reading_ease', 'dale_chall_readability_score']
features_log_minmax_transform = pd.DataFrame(data = features_raw)
features_log_minmax_transform[numerical] = scaler.fit_transform(features_raw[numerical])


# Show an example of a record with scaling applied
display(features_log_minmax_transform.head(n = 6))

,minor,contact_email,vendors,share_personal,Cookies,fog_index,avg_sentence_length,flesch_reading_ease,dale_chall_readability_score,may
0,1,0,1,0,0,0.660803,0.660803,0.332320,0.660400,1
1,0,1,0,1,1,0.418325,0.418325,0.569989,0.418331,1
2,0,0,0,0,0,0.369417,0.369417,0.617905,0.368978,1
3,1,1,0,0,1,0.590213,0.590213,0.401577,0.589894,1
4,0,0,0,0,0,0.747232,0.747232,0.295327,0.747356,1
5,0,1,0,0,1,0.259556,0.259556,0.725507,0.259694,1


In [696]:
# Import train_test_split
from sklearn.model_selection import train_test_split
# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_log_minmax_transform, 
                                                    scores, 
                                                    test_size = 0.2, 
                                                    random_state = 0)
# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 347 samples.
Testing set has 87 samples.


In [673]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=15, max_depth=31,random_state=1, criterion="entropy", max_features = 3)
forest.fit(X_train, y_train)

predictions = forest.predict(X_test)
from sklearn.metrics import f1_score
f1_score(y_test, predictions, average='macro')

0.8363103563103564

In [585]:
# for x in range(1,200):
#     for y in range(1,200):
#         for z in range(1,200):
#             forest  = RandomForestClassifier(n_estimators=y, max_depth=z,random_state=x, criterion="gini")
#             forest.fit(X_train, y_train)
#             predi = forest.predict(X_test)
#             ans = f1_score(y_test, predi, average = 'macro')
#             if ans > .92:
#                 print ("random_state: " + str(x))
#                 print ("n_estimators: " + str(y))
#                 print("max_depth: " + str(z))
#                 print(ans)

In [949]:
from sklearn.tree import DecisionTreeClassifier
dt  = DecisionTreeClassifier(random_state = 47, criterion='gini', splitter = 'random', max_depth=14, max_features = 3)
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=14,
            max_features=3, max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=47, splitter='random')

In [950]:
# for x in range(0,100):
#     for y in range(1,100):
#         for z in range(1,11):
#             dt  = DecisionTreeClassifier( presort = True,random_state = x, criterion='gini', splitter = 'random', max_depth=y, max_features = z)
#             dt.fit(X_train, y_train)
#             predi = dt.predict(X_test)
#             ans = f1_score(y_test, predi, average = 'macro')
#             if ans > .952:
#                 print ("random_state: " + str(x))
#                 print ("n_estimators: " + str(y))
#                 print("max_features: " + str(z))
#                 print(ans)

In [958]:
# dt.fit(X_train, y_train)
predictions = dt.predict(X_test)
from sklearn.metrics import f1_score 
f1_score(y_test, predictions, average = 'macro')

0.951861763131738

In [952]:
# from sklearn.tree import DecisionTreeClassifier
# dt  = DecisionTreeClassifier( random_state = 1, criterion='gini', splitter = 'random', max_depth=20)
# dt.fit(X_train, y_train)

In [959]:
# from sklearn.ensemble import AdaBoostClassifier
# #good params base_estimator = dt,n_estimators=5, learning_rate=1.0, algorithm='SAMME.R', random_state=1)

# #good params w nltk stop words model = AdaBoostClassifier(base_estimator = dt,n_estimators=14, learning_rate=.2, algorithm='SAMME.R', random_state=1)
# ada = AdaBoostClassifier(random_state = 21,base_estimator = dt,n_estimators=13, learning_rate=.2, algorithm='SAMME.R')

In [960]:
# for x in range(0,300):
#     for y in range(1,220):
#             ada = AdaBoostClassifier(random_state = x,base_estimator = ET,n_estimators=y, learning_rate=.8, algorithm='SAMME.R')
#             ada.fit(X_train, y_train)
#             predi = ada.predict(X_test)
#             ans = f1_score(y_test, predi, average = 'macro')
#             if ans > .93:
#                 print ("random_state: " + str(x))
#                 print ("n_estimators: " + str(y))
#                 print(ans)

In [962]:
# ada.fit(X_train, y_train)
# predi = ada.predict(X_test)
# f1_score(y_test, predi, average = 'macro')

In [328]:
# for x in range(0,100):
#     for y in range(1,100):
#             ada = AdaBoostClassifier(random_state = x,base_estimator = dt,n_estimators=y, learning_rate=1.5, algorithm='SAMME.R')
#             ada.fit(X_train, y_train)
#             predi = ada.predict(X_test)
#             ans = np.mean(predi == y_test)
#             if ans > .94:
#                 print ("random_state: " + str(x))
#                 print ("n_estimators: " + str(y))
#                 print(ans)

In [649]:
from sklearn.ensemble import ExtraTreesClassifier


ET = ExtraTreesClassifier(n_estimators = 35, random_state = 0, criterion='entropy')
ET.fit(X_train, y_train)
predi = ET.predict(X_test)
ans = f1_score(y_test, predictions, average = 'macro')
print(ans)

0.951861763131738


In [965]:
from sklearn import model_selection
from sklearn.ensemble import ExtraTreesClassifier
X = features_log_minmax_transform
y = scores
num_folds = 5
num_instances = len(X)
model = ET
results = model_selection.cross_val_score(model, X, y, cv=5)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

Accuracy: 76.721% (4.222%)


In [704]:
# from sklearn.ensemble import ExtraTreesClassifier

# for x in range(0,200):
#     for y in range(1,200):
#             ET = ExtraTreesClassifier(n_estimators = y, random_state = x, criterion='entropy')
#             ET.fit(X_train, y_train)
#             predi = ET.predict(X_test)
#             ans = f1_score(y_test, predictions, average = 'macro')
#             if ans > .952:
#                 print ("random_state: " + str(x))
#                 print ("n_estimators: " + str(y))
#                 print(ans)

In [705]:
dt.feature_importances_

array([0.02161278, 0.1083077 , 0.04742066, 0.05480684, 0.14829687,
       0.13838897, 0.14761765, 0.14350001, 0.16891944, 0.02112907])

In [639]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predictions))

[[15  1  0  0  0]
 [ 2 42  0  0  0]
 [ 0  0 20  1  0]
 [ 0  0  0  5  0]
 [ 0  0  0  0  1]]


In [448]:
###########XGBOOST 

In [449]:
# import xgboost as xgb

In [458]:
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test)
# # specify parameters via map
# param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'multi:softmax'}
# xgb_param = model.get_xgb_params()
# xgb_param['num_class'] = 6
# num_round = 2
# bst = xgb.train(param, dtrain, num_round)

In [732]:
from sklearn.ensemble import IsolationForest

In [915]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
params = {
        'min_child_weight': [1, 2, 3],
        'gamma': [0.25, 0.5, 1, 1.5, 2, 3,5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.4,0.6, 0.8, 1.0],
        'max_depth': list(range(10,1000))
        }

In [923]:
xgb = XGBClassifier(learning_rate =.09, n_estimators=50, objective= 'multi:softprob')



folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='f1_macro', n_jobs=6, cv=5, verbose=3, random_state=1001 )
random_search.fit(X_train, y_train)

# model.fit(X_train, y_train)
# predi = model.predict(X_test)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=6)]: Done  23 out of  25 | elapsed:    2.7s remaining:    0.1s
[Parallel(n_jobs=6)]: Done  25 out of  25 | elapsed:    2.7s finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.09, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=5, n_jobs=6,
          param_distributions={'min_child_weight': [1, 2, 3], 'gamma': [0.25, 0.5, 1, 1.5, 2, 3, 5], 'subsample': [0.6, 0.8, 1.0], 'colsample_bytree': [0.4, 0.6, 0.8, 1.0], 'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,...980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999]},
          pre_dispatch='2*n_jobs', random_state=1001, refit=True,
        

In [924]:
random_search.best_score_
# random_search.best_params_
#f1_score(y_test, predi, average = 'macro')

0.6853745622586954